In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras import optimizers
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [4]:
# import competition dataset
x_train = np.load('train_data.npy')
y_train = pd.read_csv('train_labels.csv')
y_train = np.array(y_train)
y_train = y_train[:,1]

x_test = np.load('test_data.npy')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalization 
# normalizer = Normalizer().fit(x_train)
# x_train = normalizer.transform(x_train)
# x_test=normalizer.transform(x_test)

# reshape
x_train = x_train.reshape(-1,210,210)
x_test = x_test.reshape(-1,210,210)


In [5]:
# make output a 2 dim vector indicating class
y_train = np_utils.to_categorical(y_train, 2)

In [6]:
# LSTM

from keras.layers import LSTM,Flatten,Dense,TimeDistributed, Bidirectional
lstmmodel_1 = Sequential()

lstmmodel_1.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2,
                                   input_shape=(210,210),return_sequences=True),
                              input_shape=(210,210)))
lstmmodel_1.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2,
                                   input_shape=(210,210),return_sequences=True)))


lstmmodel_1.add(Flatten())
lstmmodel_1.add(Dense(128))
lstmmodel_1.add(Dropout(0.5))
lstmmodel_1.add(Dense(2, activation='sigmoid'))

W0926 15:30:14.515105  1320 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0926 15:30:14.557046  1320 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0926 15:30:14.563039  1320 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0926 15:30:19.074234  1320 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please us

In [7]:
#model.summary()
lstmmodel_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 210, 512)          956416    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 210, 256)          656384    
_________________________________________________________________
flatten_1 (Flatten)          (None, 53760)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               6881408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 8,494,466
Trainable params: 8,494,466
Non-trainable params: 0
_________________________________________________________________


In [8]:
# compile model
adam=optimizers.Adam(lr=.0001)
lstmmodel_1.compile(loss='binary_crossentropy', 
              optimizer=adam,
              metrics=['accuracy'])

W0926 15:30:26.587421  1320 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0926 15:30:26.605934  1320 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0926 15:30:26.653809  1320 deprecation.py:323] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
from keras.callbacks import EarlyStopping
EarlyStopping = EarlyStopping(monitor='val_loss',patience=5,verbose=0,mode='auto')
lstmmodel_1.fit(x_train, y_train, batch_size=32, epochs=30, validation_split=0.1,callbacks=[EarlyStopping])

Train on 3599 samples, validate on 400 samples
Epoch 1/30
3599/3599 [==============================] - 2198s 611ms/step - loss: 0.4295 - acc: 0.8594 - val_loss: 0.2994 - val_acc: 0.9150
Epoch 2/30
3599/3599 [==============================] - 1797s 499ms/step - loss: 0.3465 - acc: 0.8776 - val_loss: 0.2765 - val_acc: 0.9225
Epoch 3/30
3599/3599 [==============================] - 1855s 516ms/step - loss: 0.2883 - acc: 0.9005 - val_loss: 0.2392 - val_acc: 0.9250
Epoch 4/30
3599/3599 [==============================] - 1334s 371ms/step - loss: 0.2538 - acc: 0.9172 - val_loss: 0.2414 - val_acc: 0.9375
Epoch 5/30
3599/3599 [==============================] - 1371s 381ms/step - loss: 0.2212 - acc: 0.9298 - val_loss: 0.2438 - val_acc: 0.9325
Epoch 6/30
3599/3599 [==============================] - 1361s 378ms/step - loss: 0.2033 - acc: 0.9340 - val_loss: 0.2361 - val_acc: 0.9375
Epoch 7/30
3599/3599 [==============================] - 1442s 401ms/step - loss: 0.1987 - acc: 0.9386 - val_loss: 0.235

In [10]:
# make predictions
y_predict= lstmmodel_1.predict_classes(x_test)

In [11]:
print(y_predict[:500])

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 

In [12]:
prediction = pd.DataFrame(y_predict,columns=['label']).to_csv('submission_1.csv')

In [13]:
#1. original//  0.2627 - acc: 0.9434 - val_loss: 0.3483 - val_acc: 0.9271
#2 //loss: 0.1336 - acc: 0.9544 - val_loss: 0.2183 - val_acc: 0.9325
#3 add another STLM layer (128, 128) //loss: 0.1749 - acc: 0.9415 - val_loss: 0.2377 - val_acc: 0.9375 (interrupt,may even higher)
#4 change dense from 32 to 128, filters (256,128) //loss: 0.1278 - acc: 0.9575 - val_loss: 0.2955 - val_acc: 0.9237